## 🦜🔗 LangChain Agent mit einem SQL Toolkit


Zuerst enpacken wir unsere Demo Datenbank mit der wir gleich arfbeiten werden.

In [ ]:
! [ ! -f northwind.db ] && bunzip2 -k northwind.db.bz2

#### Wir erstellen das SQL Toolkit und den Agenten


In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from helpers import llm

model = llm(temperature=0)
db = SQLDatabase.from_uri("sqlite:///northwind.db")
toolkit = SQLDatabaseToolkit(db=db, llm=model)

In [ ]:
from langgraph.prebuilt import create_agent_executor
from langchain.agents import create_openai_functions_agent
from langchain import hub


tools = toolkit.get_tools()
prompt: ChatPromptTemplate = hub.pull("reactagent/sql")
prompt = prompt.partial(dialect=toolkit.dialect, top_k=10)
agent_runnable = create_openai_functions_agent(model, tools, prompt)
agent_excutor = create_agent_executor(agent_runnable, tools)

prompt

#### Wir definieren eine Funktion, die die gestreamte Ausgabe des Agenten formatiert.

Man muss den folgenden Code nicht durchlesen. Man sollte sich nur merken, dass man solche Code-Brocken üblicherweise selbst erstellen muss.


In [ ]:
from typing import Any, AsyncIterator, List, Tuple
from langchain_core.agents import AgentActionMessageLog, AgentFinish


async def formatted_output_streamer(stream: AsyncIterator[Any]) -> AsyncIterator[Any]:
    async for chunk in stream:
        output = ""
        for key, value in chunk.items():
            if key == "agent":
                outcome = value.get("agent_outcome")
                if isinstance(outcome, AgentActionMessageLog):
                    output += f"Agent log:\n\n{outcome.log.strip()}"
                elif isinstance(outcome, AgentFinish):
                    output += f"Agent finished:\n\n{outcome.log.strip()}"
                output += "\n\n----------------------------------------------------------------------------------------\n\n"
            elif key == "action":
                steps: List[Tuple[AgentActionMessageLog, str]] = value.get(
                    "intermediate_steps"
                )
                for index, step in enumerate(steps):
                    output += f"Tool log:\n\n{step[1].strip()}"
                    if index < len(steps) - 1:
                        print("----------------")
                output += "\n\n----------------------------------------------------------------------------------------\n\n"
            elif key == "__end__":
                output = "Done"
        yield output

#### Wir pipen (chainen) den Agenten mit dem Formatierer


In [ ]:
app = agent_excutor | formatted_output_streamer

#### Looos....


In [ ]:
inputs = {"input": "Where do i find the orders?"}
async for chunk in app.astream(inputs):
    print(chunk)

In [ ]:
inputs = {"input": "Which Employee has the most orders?"}
async for chunk in app.astream(inputs):
    print(chunk)

#### Schauen wir mal, ob er das hier hinbekommt.


In [ ]:
inputs = {
    "input": "Which Customer has had the Order with the highest total cost ever? What was the Order Id?"
}
async for chunk in app.astream(inputs):
    print(chunk)

#### Und noch einmal das batchen demonstrieren.


In [ ]:
async def formatted_output_batcher(item: Any) -> str:
    return [item.get("input"), item.get("agent_outcome").return_values.get("output")]


batcher = agent_excutor | formatted_output_batcher

result = await batcher.abatch(
    [
        {"input": "Where do i find the orders?"},
        {"input": "Which Employee has the most orders?"},
        {
            "input": "Which Customer has had the Order with the highest total cost ever? What was the Order Id?"
        },
    ]
)

for index, item in enumerate(result):
    print(f"Query {index+1}:")
    print(f"Question: {item[0]}")
    print(f"Answer: {item[1]}\n\n")